In [1]:
import os
import re
import glob
import json
import data 
import requests
import pandas as pd
from io import StringIO

In [2]:
# with open(r'C:\Users\User\Desktop\py_scripts\airflow_tests\json_hits_constant.json', 'r', encoding="utf-8") as jsonFile1:
#     constantJsonData = json.load(jsonFile1)

start_date_v = '2024-07-01'
end_date_v = '2024-07-01'
#col_tab_v = constantJsonData['YaApiHitsConstant']['col_tab']
counter_id_v = '12084547'
API_token_v = 'y0_AgAAAAB29aj2AAwCXgAAAAEJFTfLAACPMhg8YaZLl6Rcw-kzM1TBWecfyg'
headers_v = {'Authorization': f"OAuth {API_token_v}", 'Accept-Encoding': 'gzip'}
path_request_id_v = r'C:\Users\User\Desktop\py_scripts\tsv\tmp\request_id_folder\request_id.txt'
path_parts_and_partsizes_txt_v = r'C:\Users\User\Desktop\py_scripts\tsv\tmp\parts_and_partsizes_folder\parts_and_partsizes.txt'
path_tsv_v = r'C:\Users\User\Desktop\py_scripts\tsv\tmp\tsv_folder'
path_parquet_v = r'C:\Users\User\Desktop\py_scripts\tsv\tmp\parquet_folder'

In [28]:
# Оценка возможности создания запроса

def ocenka_vozmozhnisti_sozd_zaprosa(counter_id, API_token, headers, start_date, end_date, col_tab):
    get_ocenka = f"https://api-metrika.yandex.net/management/v1/counter/{counter_id}/logrequests/evaluate"

    params_ocenka = {
        'date1': start_date,
        'date2': end_date,
        'fields': col_tab,
        'source': 'hits'  # hits visits 
        }

    r = requests.get(get_ocenka, params = params_ocenka, headers=headers)

    if r.status_code == 200: 
        if r.json()['log_request_evaluation']['possible'] == True:
            print('Выгрузка возможна')
        elif r.json()['log_request_evaluation']['possible'] == False:
            print('YaApi ответил что выгрузка невозможна')
#            print(r.json())
        else: print('Что-то пошло не так')
    else: 
        print('Запрос упал: ', r.status_code)
        return('Error')

In [29]:
%%time
ocenka_vozmozhnisti_sozd_zaprosa(counter_id=counter_id_v, API_token=API_token_v, headers=headers_v,
                                 start_date=start_date_v, end_date=end_date_v, col_tab=col_tab_v)

Выгрузка возможна
CPU times: total: 31.2 ms
Wall time: 7.88 s


In [30]:
# Создание запроса логов
# Нужно сохранить request_id

def sozd_zaprosa_logov(counter_id, API_token, headers, start_date, end_date, col_tab, path_request_id):
    post_sozdanie_zaprosa_logov = f"https://api-metrika.yandex.net/management/v1/counter/{counter_id}/logrequests"

    params_sozdanie_zaprosa_logov = {
        'date1': start_date,
        'date2': end_date,
        'fields': col_tab,
        'source': 'hits'  # hits visits 
    }

    r = requests.post(post_sozdanie_zaprosa_logov, params = params_sozdanie_zaprosa_logov, headers=headers)

    if r.status_code == 200: 
        if r.json()['log_request']['status'] == 'created': 
            print('Запрос создаётся')
            #print(r.json()['log_request']['request_id'])
            with open(path_request_id, 'w', encoding='utf-8') as f:
                f.write(str(r.json()['log_request']['request_id']))
        else: print('Что-то пошло не так')
    else: 
        print('Запрос упал: ', r.status_code)
        return('Error')
        
# Функция нужна чтоб читать сохраненный request_id
def read_request_id(path_request_id):
    with open(path_request_id, "r") as f:
        request_id = f.read()
    return request_id

In [31]:
%%time
sozd_zaprosa_logov(counter_id=counter_id_v, API_token=API_token_v, headers=headers_v, start_date=start_date_v, end_date=end_date_v,
                   col_tab=col_tab_v, path_request_id=path_request_id_v)

Запрос создаётся
CPU times: total: 15.6 ms
Wall time: 521 ms


In [32]:
%%time
read_request_id(path_request_id=path_request_id_v)

CPU times: total: 0 ns
Wall time: 1.56 ms


'38107832'

In [3]:
# Функция используется в def get_status для записи "частей" выгрузки и размеров этих частей

def extract_parts_and_partsizes(data, path): 
    if os.path.exists(path) == True: 
        os.remove(path)
    else: pass
    for i in range(0, len(data)): 
        #print(data[i])
        with open(path, 'a', encoding='utf-8') as f:
            f.write(str(data[i]['part_number']) + '|' + str(data[i]['size']) + '\n')


# Информация о запросе логов.
# Лог со статусом processed готов к выгрузке.
# Если ваш запрос находится в статусе created, дождитесь его выполнения.

def get_status(counter_id, API_token, headers, request_id, path_parts_and_partsizes_txt): 
    get_status = f"https://api-metrika.yandex.net/management/v1/counter/{counter_id}/logrequest/{request_id}"

    r = requests.get(get_status, headers=headers)
    
    if r.status_code == 200: 
        if r.json()['log_request']['status'] == 'processed': 
            print('Нужно выгружать')
            extract_parts_and_partsizes(data=r.json()['log_request']['parts'], path=path_parts_and_partsizes_txt)
            print(r.json())
        elif r.json()['log_request']['status'] == 'created':
            print('Нужно ждать, запрос в статусе created и уже готовится или в очереди на исполнение')
        else: 
            print('Что-то пошло не так ', r.status_code)
            print(r.json())
    else: 
        print('Запрос упал: ', r.status_code) 
        return('Error')

In [4]:
%%time
get_status(counter_id=counter_id_v, API_token=API_token_v, headers=headers_v, path_parts_and_partsizes_txt=path_parts_and_partsizes_txt_v, 
           request_id='38244344')

Что-то пошло не так  200
{'log_request': {'request_id': 38244344, 'counter_id': 12084547, 'source': 'visits', 'date1': '2024-07-04', 'date2': '2024-07-04', 'fields': ['ym:s:visitID', 'ym:s:date', 'ym:s:eventsProductBrand', 'ym:s:eventsProductCategory', 'ym:s:eventsProductCategory1', 'ym:s:eventsProductCategory2', 'ym:s:eventsProductCategory3', 'ym:s:eventsProductCategory4', 'ym:s:eventsProductCoupon', 'ym:s:eventsProductCurrency', 'ym:s:eventsProductDiscount', 'ym:s:eventsProductEventTime', 'ym:s:eventsProductID', 'ym:s:eventsProductList', 'ym:s:eventsProductName', 'ym:s:eventsProductPosition', 'ym:s:eventsProductPrice', 'ym:s:eventsProductQuantity', 'ym:s:eventsProductType', 'ym:s:eventsProductVariant', 'ym:s:impressionsDateTime', 'ym:s:impressionsProductBrand', 'ym:s:impressionsProductCategory', 'ym:s:impressionsProductCategory1', 'ym:s:impressionsProductCategory2', 'ym:s:impressionsProductCategory3', 'ym:s:impressionsProductCategory4', 'ym:s:impressionsProductCoupon', 'ym:s:impressi

In [25]:
# Загрузка частей логов обработанного запроса

def dowload_parts(counter_id, API_token, headers, request_id, path_parts_and_partsizes_txt, path_tsv):
    with open(path_parts_and_partsizes_txt, "r") as f:
        file_parts = f.readlines()
    #print(len(file_parts))
    for i in range(0, len(file_parts)):
        big_str = (file_parts[i].split('|'))
        part_number = str(big_str[0])
        size = int(re.sub('[^A-Za-z0-9]+', '', big_str[1]))
        #print(str(part) + '_' + str(size))

        get_dowload_part = f"https://api-metrika.yandex.net/management/v1/counter/{counter_id}/logrequest/{request_id}/part/{part_number}/download" 
                             
        
        r_dowload_part = requests.get(get_dowload_part, headers=headers)

        if r_dowload_part.status_code == 200: 
            s=str(r_dowload_part.content,'utf-8')
    
            if len(s.encode('utf-8')) >= size:
                data = StringIO(s) 
                df=pd.read_csv(data, sep="\t")
                path_tsv_final = path_tsv + r"\tsv_" + str(request_id) + "_" + part_number + ".csv"
                df.to_csv(path_tsv_final, sep='\t', encoding='utf-8', index=False, header=True)
            # надо придумать нормальное исключение
            else: 
                print("Пришёл битый файл: ", big_str)
                return("Error")
        else:        
            print('Запрос упал: ', r.status_code) 
            return('Error')

In [24]:
%%time
dowload_parts(counter_id=counter_id_v, API_token=API_token_v, headers=headers_v, request_id='38107784',
              path_parts_and_partsizes_txt=path_parts_and_partsizes_txt_v, path_tsv=path_tsv_v)

NameError: name 'dowload_parts' is not defined

In [ ]:
# Конвертируем csv в parquet
def csv_2_parquet(path_csv_folder, path_parquet, request_id):
    all_files = glob.glob(os.path.join(path_csv_folder, "*.csv"))
    #print(all_files)
    for i, j in zip(all_files, range(0, len(all_files))):
        df = pd.read_csv(i, sep="\t", encoding='utf-8')
        path_parquet_name = path_parquet + r"\\parquet_" + request_id + "_" + str(j) + ".parquet"
        df.to_parquet(path_parquet_name)

In [ ]:
%%time
csv_2_parquet(path_csv_folder=path_tsv_v, path_parquet=path_parquet_v,
              request_id=read_request_id(path_request_id=path_request_id_v))

In [5]:
# Очистка подготовленных для загрузки логов обработанного запроса по request_id

def clear_YaApi_query(counter_id, API_token, headers, request_id):
    
    post_clean = f"https://api-metrika.yandex.net/management/v1/counter/{counter_id}/logrequest/{request_id}/clean"
    r = requests.post(post_clean, headers=headers)
    if r.status_code == 200: 
        if r.json()['log_request']['status'] == 'cleaned_by_user':
            print('Данные Яндекс метрики успешно очищены на стороне Яндекса')
        else: 
            print('Что-то пошло не так')
            print(r.json())
    else:        
        print('Запрос упал: ', r.status_code) 
        return('Error')

In [6]:
#%%time

clear_YaApi_query(counter_id=counter_id_v, API_token=API_token_v, headers=headers_v,
                  request_id='38208606')

Данные Яндекс метрики успешно очищены на стороне Яндекса


In [ ]:
# Удаляет все файлы из папкм tmp (включая parquet)

def del_tmp(path_request_id, path_parts_and_partsizes_txt, path_tsv, path_parquet):
    os.remove(path_request_id)
    os.remove(path_parts_and_partsizes_txt)
    all_csv_files = glob.glob(os.path.join(path_tsv, "*.csv"))
    for i in all_csv_files:
        os.remove(i)
    all_parquet_files = glob.glob(os.path.join(path_parquet, "*.parquet"))    
    for j in all_parquet_files:
        os.remove(j)
    print("Все файлы удалены из папкм tmp")

In [ ]:
%%time
del_tmp(path_request_id=path_request_id_v, path_parts_and_partsizes_txt=path_parts_and_partsizes_txt_v,
        path_tsv=path_tsv_v, path_parquet=path_parquet_v)

In [ ]:
# path_1 = r"C:\Users\User\Desktop\py_scripts\tsv\tmp\tsv_folder\tsv_36549344_0.csv"
# path_2 = r"C:\Users\User\Desktop\py_scripts\tsv\tmp\36549344_final.csv"
# df_1 = pd.read_csv(path_1, sep="\t")
# df_2 = pd.read_csv(path_2, sep="\t")
# print(len(df_1), " | ", len(df_2))

In [ ]:
# path_p = r"C:\Users\User\Desktop\py_scripts\tsv\tmp\36549344_final.parquet"
# df_p = pd.read_parquet(path_p) 
# df_p.head(5)